<a href="https://colab.research.google.com/github/ambarja/utilsUPCH/blob/main/00_get_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **ERA5Land and Chirps weighted by Population**

In [4]:
import ee
import geemap
import geopandas as gpd
import pandas as pd
import os

In [ ]:
# Authenticate in Google Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-geografope')

In [ ]:
# District dataset original without simplify
dist = ee.FeatureCollection('users/geografo2023/distritos_inei_2017')

In [ ]:
# Define the output of dataset
out_dir = os.path.join(os.path.expanduser("~"), "/content/drive/MyDrive/paloma")

### **1. Working with Population data of WorldPop**

In [ ]:
# Population dataset for Peru - WorldPop
pop_2000 = ee.Image('WorldPop/GP/100m/pop/PER_2000').rename('pop_2000');pop_2001 = ee.Image('WorldPop/GP/100m/pop/PER_2001').rename('pop_2001')
pop_2002 = ee.Image('WorldPop/GP/100m/pop/PER_2002').rename('pop_2002');pop_2003 = ee.Image('WorldPop/GP/100m/pop/PER_2003').rename('pop_2003')
pop_2004 = ee.Image('WorldPop/GP/100m/pop/PER_2004').rename('pop_2004');pop_2005 = ee.Image('WorldPop/GP/100m/pop/PER_2005').rename('pop_2005')
pop_2006 = ee.Image('WorldPop/GP/100m/pop/PER_2006').rename('pop_2006');pop_2007 = ee.Image('WorldPop/GP/100m/pop/PER_2007').rename('pop_2007')
pop_2008 = ee.Image('WorldPop/GP/100m/pop/PER_2008').rename('pop_2008');pop_2009 = ee.Image('WorldPop/GP/100m/pop/PER_2009').rename('pop_2009')
pop_2010 = ee.Image('WorldPop/GP/100m/pop/PER_2010').rename('pop_2010');pop_2011 = ee.Image('WorldPop/GP/100m/pop/PER_2011').rename('pop_2011')
pop_2012 = ee.Image('WorldPop/GP/100m/pop/PER_2012').rename('pop_2012');pop_2013 = ee.Image('WorldPop/GP/100m/pop/PER_2013').rename('pop_2013')
pop_2014 = ee.Image('WorldPop/GP/100m/pop/PER_2014').rename('pop_2014');pop_2015 = ee.Image('WorldPop/GP/100m/pop/PER_2015').rename('pop_2015')
pop_2016 = ee.Image('WorldPop/GP/100m/pop/PER_2016').rename('pop_2016');pop_2017 = ee.Image('WorldPop/GP/100m/pop/PER_2017').rename('pop_2017')
pop_2018 = ee.Image('WorldPop/GP/100m/pop/PER_2018').rename('pop_2018');pop_2019 = ee.Image('WorldPop/GP/100m/pop/PER_2019').rename('pop_2019')
pop_2020 = ee.Image('WorldPop/GP/100m/pop/PER_2020').rename('pop_2020');pop_2021 = ee.Image('WorldPop/GP/100m/pop/PER_2020').rename('pop_2021')
pop_2022 = ee.Image('WorldPop/GP/100m/pop/PER_2020').rename('pop_2022');pop_2023 = ee.Image('WorldPop/GP/100m/pop/PER_2020').rename('pop_2023')

In [ ]:
# New ImageCollection for Peru (2000-2023)
pop_total = ee.ImageCollection([
    pop_2000,pop_2001,pop_2002,pop_2003,pop_2004,pop_2005,pop_2006,pop_2007,pop_2008,pop_2009,pop_2010,pop_2011,
    pop_2012,pop_2013,pop_2014,pop_2015,pop_2016,pop_2017,pop_2018,pop_2019,pop_2020,pop_2021,pop_2022,pop_2023]).toBands()

In [ ]:
# Downloading Population dataset from EE
out_pop_stats = os.path.join(out_dir, "population_2000_2023.csv")
geemap.zonal_stats(pop_total, dist, out_pop_stats, stat_type="SUM", scale=100)

### **2. Working with Chirps data on a monthly scale**

In [ ]:
# List of years
start_year = 2000
end_year = 2023
scale = 100
stats = "MEAN"

In [ ]:
# Original dataset of  Chirps and WorldPop
chirps_db = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY").select('precipitation')
world_pop = ee.ImageCollection('WorldPop/GP/100m/pop')
inicio = start_year
final = end_year + 1
list_of_years = range(inicio,final)

# New functions for calculating the weighted variable
def yearly_monthly_mean(year):
  """Function to map the monthly_sum() to every month in the months sequences."""
  def monthly_sum(month):
    """Function to calculate the sum of rainy days for a given month in a given year."""
    w = chirps_db\
        .filter(ee.Filter.calendarRange(year, year, 'year')) \
        .filter(ee.Filter.calendarRange(month, month, 'month')) \
        .mean() # The mean of rain in a specific month

    pob = world_pop.filter(ee.Filter.calendarRange(year,year, 'year')).mosaic()
    weighted = w.multiply(pob)
    return weighted.set('year', year).set('month', month).set('system:time_start', ee.Date.fromYMD(year, month, 1))
  return months.map(monthly_sum)

# New functions for computate the weighted variable
def yearly_monthly_mean_2021_2023(year):
  """Function to map the monthly_sum() to every month in the months sequences."""
  def monthly_sum(month):
    """Function to calculate the sum of rainy days for a given month in a given year."""
    w = chirps_db\
        .filter(ee.Filter.calendarRange(year, year, 'year')) \
        .filter(ee.Filter.calendarRange(month, month, 'month')) \
        .mean() # The mean of rain in a specific month
    pob = world_pop.filter(ee.Filter.calendarRange(2020,2020, 'year')).mosaic()
    weighted = w.multiply(pob)
    return weighted.set('year', year).set('month', month).set('system:time_start', ee.Date.fromYMD(year, month, 1))
  return months.map(monthly_sum)

for i in list_of_years:
  if i <= 2020:
    # Defined year
    year = i
    # List for map process in EE
    years = ee.List.sequence(year,year); months = ee.List.sequence(1,12)
    # Preprocessing datate in EE
    img = ee.ImageCollection.fromImages(years.map(yearly_monthly_mean).flatten()).toBands()
    # Output data processed
    output = os.path.join(out_dir,f'chirps_monthly_precip_weighted_pob_{year}.csv')
    geemap.zonal_stats(img, dist, output, stat_type= stats, scale = scale)
    print('dataset exported!')

  else:
    # Defined year
    year = i
    # List for map process in EE
    years = ee.List.sequence(year,year); months = ee.List.sequence(1,12)
    # Preprocessing datate in EE
    img = ee.ImageCollection.fromImages(years.map(yearly_monthly_mean_2021_2023).flatten()).toBands()
    # Output data processed
    output = os.path.join(out_dir,f'chirps_monthly_precip_weighted_pob_{year}.csv')
    geemap.zonal_stats(img, dist, output, stat_type= stats, scale = scale)
    print('dataset exported!')

### **4. Working with ERA5Land data on a monthly scale**

In [71]:
# List of years
start_year = 2000
end_year = 2023
scale = 100
stats = "MEAN"
variable = "temperature_2m_min"

In [ ]:
# Original dataset of  Era5land and WorldPop
era5land = ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY_AGGR')
world_pop = ee.ImageCollection('WorldPop/GP/100m/pop')
inicio = start_year
final = end_year + 1
list_of_years = range(inicio,final)

# New functions for computate the weighted variable
def yearly_monthly_mean(year):
  """Function to map the monthly_sum() to every month in the months sequences."""
  def monthly_sum(month):
    """Function to calculate the sum of rainy days for a given month in a given year."""
    w = era5land\
        .select(variable)\
        .filter(ee.Filter.calendarRange(year, year, 'year')) \
        .filter(ee.Filter.calendarRange(month, month, 'month')) \
        .mean() # The mean of rain in a specific month

    pob = world_pop.filter(ee.Filter.calendarRange(year,year, 'year')).mosaic()
    weighted = w.multiply(pob)
    return weighted.set('year', year).set('month', month).set('system:time_start', ee.Date.fromYMD(year, month, 1))
  return months.map(monthly_sum)

# New functions for computate the weighted variable
def yearly_monthly_mean_2021_2023(year):
  """Function to map the monthly_sum() to every month in the months sequences."""
  def monthly_sum(month):
    """Function to calculate the sum of rainy days for a given month in a given year."""
    w = era5land\
        .select(variable)\
        .filter(ee.Filter.calendarRange(year, year, 'year')) \
        .filter(ee.Filter.calendarRange(month, month, 'month')) \
        .mean() # The mean of rain in a specific month
    pob = world_pop.filter(ee.Filter.calendarRange(2020,2020, 'year')).mosaic()
    weighted = w.multiply(pob)
    return weighted.set('year', year).set('month', month).set('system:time_start', ee.Date.fromYMD(year, month, 1))
  return months.map(monthly_sum)

for i in list_of_years:
  if i <= 2020:
    # Defined year
    year = i
    # List for map process in EE
    years = ee.List.sequence(year,year); months = ee.List.sequence(1,12)
    # Preprocessing datate in EE
    img = ee.ImageCollection.fromImages(years.map(yearly_monthly_mean).flatten()).toBands()
    # Output data processed
    output = os.path.join(out_dir,f'era5land_monthly_{variable}_weighted_pob_{year}.csv')
    geemap.zonal_stats(img, dist, output, stat_type= stats, scale = scale)
    print('dataset exported!')

  else:
    # Defined year
    year = i
    # List for map process in EE
    years = ee.List.sequence(year,year); months = ee.List.sequence(1,12)
    # Preprocessing datate in EE
    img = ee.ImageCollection.fromImages(years.map(yearly_monthly_mean_2021_2023).flatten()).toBands()
    # Output data processed
    output = os.path.join(out_dir,f'era5land_monthly_{variable}_weighted_pob_{year}.csv')
    geemap.zonal_stats(img, dist, output, stat_type= stats, scale = scale)
    print('dataset exported!')

### **8. Working with ERA5Land - Tmean**

In [ ]:
# List of years
start_year = 2000
end_year = 2023
scale = 100
stats = "MEAN"
variable_max = "temperature_2m_max"
variable_min = "temperature_2m_min"

In [ ]:
# Original dataset of  Era5land and WorldPop
era5land = ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY_AGGR')
world_pop = ee.ImageCollection('WorldPop/GP/100m/pop')
inicio = start_year
final = end_year + 1
list_of_years = range(inicio,final)

# New functions for computate the weighted variable
def yearly_monthly_mean(year):
  """Function to map the monthly_sum() to every month in the months sequences."""
  def monthly_sum(month):
    """Function to calculate the sum of rainy days for a given month in a given year."""

    # Maximun value
    w_max = era5land\
        .select(variable_max)\
        .filter(ee.Filter.calendarRange(year, year, 'year')) \
        .filter(ee.Filter.calendarRange(month, month, 'month')) \
        .mean() # The mean of rain in a specific month

    # Minimum value
    w_min = era5land\
        .select(variable_min).filter(ee.Filter.calendarRange(year, year, 'year')) \
        .filter(ee.Filter.calendarRange(month, month, 'month')) \
        .mean()\
        .subtract(273.15)

    # Mean Value
    w_mean = w_max.add(w_min).divide(ee.Image(2)).rename('tmean')

    # Population value
    pob = world_pop.filter(ee.Filter.calendarRange(year,year, 'year')).mosaic()

    # Weighted variable
    w_pob = w_mean.multiply(pob)

    return w_pob.set('year', year).set('month', month).set('system:time_start', ee.Date.fromYMD(year, month, 1))
  return months.map(monthly_sum)

# New functions for computate the weighted variable
def yearly_monthly_mean_2021_2023(year):
  """Function to map the monthly_sum() to every month in the months sequences."""
  def monthly_sum(month):
    """Function to calculate the sum of rainy days for a given month in a given year."""

    # Maximun value
    w_max = era5land\
        .select(variable_max)\
        .filter(ee.Filter.calendarRange(year, year, 'year')) \
        .filter(ee.Filter.calendarRange(month, month, 'month')) \
        .mean() # The mean of rain in a specific month

    # Minimum value
    w_min = era5land\
        .select(variable_min).filter(ee.Filter.calendarRange(year, year, 'year')) \
        .filter(ee.Filter.calendarRange(month, month, 'month')) \
        .mean()\
        .subtract(273.15)

    # Mean Value
    w_mean = w_max.add(w_min).divide(ee.Image(2)).rename('tmean')

    # Population value
    pob = world_pop.filter(ee.Filter.calendarRange(2020,2020, 'year')).mosaic()

    # Weighted variable
    w_pob = w_mean.multiply(pob)

    return w_pob.set('year', year).set('month', month).set('system:time_start', ee.Date.fromYMD(year, month, 1))
  return months.map(monthly_sum)

for i in list_of_years:
  if i <= 2020:
    # Defined year
    year = i
    # List for map process in EE
    years = ee.List.sequence(year,year); months = ee.List.sequence(1,12)
    # Preprocessing datate in EE
    img = ee.ImageCollection.fromImages(years.map(yearly_monthly_mean).flatten()).toBands()
    # Output data processed
    output = os.path.join(out_dir,f'era5land_monthly_temperature_2m_mean_weighted_pob_{year}.csv')
    geemap.zonal_stats(img, dist, output, stat_type= stats, scale = scale)
    print('dataset exported!')

  else:
    # Defined year
    year = i
    # List for map process in EE
    years = ee.List.sequence(year,year); months = ee.List.sequence(1,12)
    # Preprocessing datate in EE
    img = ee.ImageCollection.fromImages(years.map(yearly_monthly_mean_2021_2023).flatten()).toBands()
    # Output data processed
    output = os.path.join(out_dir,f'era5land_monthly_temperature_2m_mean_weighted_pob_{year}.csv')
    geemap.zonal_stats(img, dist, output, stat_type= stats, scale = scale)
    print('dataset exported!')